# About this notebook
this notebook is written by changhyun yi, taking Modulab's Aiffel Program (Deep learning education course).  
이 노트북은 모두의연구소 AIFFEL 과정 중 going-deeper node 16, hugging face custom model 만들기에 제출하기 위해 만들었습니다.

# mnli task
MNLI task에 대한 description은 [MultiNLI](https://cims.nyu.edu/~sbowman/multinli/) 에서 찾을 수 있습니다. 두 문장이 __전제(premise)__ 와 __가설(hypothesis)__ 로 제시되고 그 사이의 관계를 entailment - 이어짐, contradict - 반(反)함, neutral - 관계없음 세 label로 나눕니다. 

node에서 다룬 것은 정제된 BERT pretrain모델입니다. 이것은 label없이 말뭉치에서 마스크를 씌우고 문장이 이어지는지를 라벨로 하여 학습시킨 것이므로, 두 문장의 관계를 따지는 것이라 연관이 아예 없어보이지는 않는데 안된다니 아쉽네요.  


# MNLI dataset 둘러보기

tensorflow-datasets 패키지와 포함된 datasets에 리스트에 대해서는 https://www.tensorflow.org/datasets/catalog/overview#all_datasets, 
그 중 GLUE에 관한 부분은 https://www.tensorflow.org/datasets/catalog/glue 에서 볼 수 있습니다.

```Shell
pip install tensorflow-datasets -U
```
glue는 다음과 같은 구조로 돼있습니다.
```python
FeaturesDict({
    'hypothesis': Text(shape=(), dtype=tf.string),
    'idx': tf.int32,
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=3),
    'premise': Text(shape=(), dtype=tf.string),
})
```

In [30]:
import tensorflow_datasets as tfds
import tensorflow as tf

In [5]:
print(tfds.__version__)

4.4.0


In [21]:
ds=tfds.load('glue/mnli')

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/5 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/392702 [00:00<?, ? examples/s]

...

Generating test_matched examples...:   0%|          | 0/9796 [00:00<?, ? examples/s]

Shuffling glue-test_matched.tfrecord...:   0%|          | 0/9796 [00:00<?, ? examples/s]

Generating test_mismatched examples...:   0%|          | 0/9847 [00:00<?, ? examples/s]

Shuffling glue-test_mismatched.tfrecord...:   0%|          | 0/9847 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /aiffel/tensorflow_datasets/glue/mnli/2.0.0. Subsequent calls will reuse this data.


In [35]:
type(tfds.load('glue/mnli'))

dict

STEP 1. mnli 데이터셋을 분석해 보기
tensorflow-datasets를 이용하여 glue/mnli를 다운로드하려면 tensorflow-datasets 라이브러리 버전을 올려야 합니다.

$ pip install tensorflow-datasets -U
위 명령어를 통해 라이브러리 업그레이드를 진행해 주세요!

STEP 2. MNLIProcessor클래스 구현하기
STEP 3. 위에서 구현한 processor 및 Huggingface에서 제공하는 tokenizer를 활용하여 데이터셋 구성하기
STEP 4. model을 생성하여 학습 및 테스트를 진행해 보기
💡 힌트
혹시 STEP 2의 진행에 어려움을 겪고 계신다면 transformer 프로젝트 내부를 살펴보시면 참고할만한 예시 코드를 찾아볼 수 있을 것입니다. transformers의 공식 github을 참고하는 것도 좋은 방법이에요!

In [38]:
ds.keys()

dict_keys([Split('train'), 'validation_matched', 'validation_mismatched', 'test_matched', 'test_mismatched'])

In [25]:
ds['train']

<PrefetchDataset shapes: {hypothesis: (), idx: (), label: (), premise: ()}, types: {hypothesis: tf.string, idx: tf.int32, label: tf.int64, premise: tf.string}>

In [56]:

for idx, doc in enumerate(ds['train']):
    print(idx,doc['idx'], doc['hypothesis'], doc['label'], doc['premise'], sep='\t')
    print('\n')
    if idx==3:
        break


0	tf.Tensor(16399, shape=(), dtype=int32)	tf.Tensor(b'Meaningful partnerships with stakeholders is crucial.', shape=(), dtype=string)	tf.Tensor(1, shape=(), dtype=int64)	tf.Tensor(b'In recognition of these tensions, LSC has worked diligently since 1995 to convey the expectations of the State Planning Initiative and to establish meaningful partnerships with stakeholders aimed at fostering a new symbiosis between the federal provider and recipients of legal services funding.', shape=(), dtype=string)


1	tf.Tensor(206287, shape=(), dtype=int32)	tf.Tensor(b'The Clinton followers kept to the higher ground in the discussion. ', shape=(), dtype=string)	tf.Tensor(0, shape=(), dtype=int64)	tf.Tensor(b'The Clinton surrogates also held the high ground in the context war.', shape=(), dtype=string)


2	tf.Tensor(352707, shape=(), dtype=int32)	tf.Tensor(b'Women have jobs in all areas of the workforce, they are almost getting the same wages as most men,', shape=(), dtype=string)	tf.Tensor(1, shape=(

문장을 몇 개 살펴보니, label이 다음과 같음을 알 수 있습니다.   

|Label|0|1|2|
|---|---|---|---|
|관계|이어짐|무관|반례|

그런데 0, 1이 확실하지 않습니다. 잘 모르겠네요. 일단 계속 해봅시다. 

[tensorflow.org, tfds.as_dataframe](https://www.tensorflow.org/datasets/api_docs/python/tfds/as_dataframe) 을 통해 pandas 데이터프레임을 받을 수 있다고 합니다.

In [48]:
keys=list(ds.keys())
print(keys)

[Split('train'), 'validation_matched', 'validation_mismatched', 'test_matched', 'test_mismatched']


In [50]:
ds[keys[0]]

<PrefetchDataset shapes: {hypothesis: (), idx: (), label: (), premise: ()}, types: {hypothesis: tf.string, idx: tf.int32, label: tf.int64, premise: tf.string}>

In [51]:
df=tfds.as_dataframe(ds[keys[0]])

In [52]:
df[:5]

,hypothesis,idx,label,premise
0,b'Meaningful partnerships with stakeholders is...,16399,1,"b'In recognition of these tensions, LSC has wo..."
1,b'The Clinton followers kept to the higher gro...,206287,0,b'The Clinton surrogates also held the high gr...
2,b'Women have jobs in all areas of the workforc...,352707,1,"b""um-hum because women are in every field now ..."
3,b'Houston is freezing and dry right now.',372070,2,b'Houston is really humid now'
4,"b""But they wouldn't be leaving right now. """,160184,1,b'But not now.'


In [53]:
len(df)

392702

용량이 300MB 정도라서 시간이 꽤 걸리므로 일부만 써서 테스트하겠습니다.

In [54]:
df=df[:1000]

In [55]:
df

,hypothesis,idx,label,premise
0,b'Meaningful partnerships with stakeholders is...,16399,1,"b'In recognition of these tensions, LSC has wo..."
1,b'The Clinton followers kept to the higher gro...,206287,0,b'The Clinton surrogates also held the high gr...
2,b'Women have jobs in all areas of the workforc...,352707,1,"b""um-hum because women are in every field now ..."
3,b'Houston is freezing and dry right now.',372070,2,b'Houston is really humid now'
4,"b""But they wouldn't be leaving right now. """,160184,1,b'But not now.'
...,...,...,...,...
995,b'John made me light the gas.',88879,2,"b'John strode across the room, and lit the gas. '"
996,"b""Music doesn't identify good or bad things. """,257511,2,"b""yeah i'm kind of thinking that's maybe our g..."
997,b'Lawrence Cavendish woke me up during the nig...,262058,0,b'It seemed to be the middle of the night when...
998,b'This was from another student at state.',201141,0,b'well the interesting thing was this was from...


In [57]:
data, info = tfds.load('glue/mnli', with_info=True)

In [58]:
data

{'train': <PrefetchDataset shapes: {hypothesis: (), idx: (), label: (), premise: ()}, types: {hypothesis: tf.string, idx: tf.int32, label: tf.int64, premise: tf.string}>,
 'validation_matched': <PrefetchDataset shapes: {hypothesis: (), idx: (), label: (), premise: ()}, types: {hypothesis: tf.string, idx: tf.int32, label: tf.int64, premise: tf.string}>,
 'validation_mismatched': <PrefetchDataset shapes: {hypothesis: (), idx: (), label: (), premise: ()}, types: {hypothesis: tf.string, idx: tf.int32, label: tf.int64, premise: tf.string}>,
 'test_matched': <PrefetchDataset shapes: {hypothesis: (), idx: (), label: (), premise: ()}, types: {hypothesis: tf.string, idx: tf.int32, label: tf.int64, premise: tf.string}>,
 'test_mismatched': <PrefetchDataset shapes: {hypothesis: (), idx: (), label: (), premise: ()}, types: {hypothesis: tf.string, idx: tf.int32, label: tf.int64, premise: tf.string}>}

In [59]:
info

tfds.core.DatasetInfo(
    name='glue',
    full_name='glue/mnli/2.0.0',
    description="""
    GLUE, the General Language Understanding Evaluation benchmark
    (https://gluebenchmark.com/) is a collection of resources for training,
    evaluating, and analyzing natural language understanding systems.
    """,
    config_description="""
    The Multi-Genre Natural Language Inference Corpus is a crowdsourced
    collection of sentence pairs with textual entailment annotations. Given a premise sentence
    and a hypothesis sentence, the task is to predict whether the premise entails the hypothesis
    (entailment), contradicts the hypothesis (contradiction), or neither (neutral). The premise sentences are
    gathered from ten different sources, including transcribed speech, fiction, and government reports.
    We use the standard test set, for which we obtained private labels from the authors, and evaluate
    on both the matched (in-domain) and mismatched (cross-domain) section. We a

In [61]:
import datasets
from datasets import load_dataset

huggingface_mnli_dataset = load_dataset('glue', 'mnli')
print(huggingface_mnli_dataset)

  0%|          | 0/5 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9847
    })
})


In [63]:
type(huggingface_mnli_dataset), huggingface_mnli_dataset

(datasets.dataset_dict.DatasetDict,
 DatasetDict({
     train: Dataset({
         features: ['premise', 'hypothesis', 'label', 'idx'],
         num_rows: 392702
     })
     validation_matched: Dataset({
         features: ['premise', 'hypothesis', 'label', 'idx'],
         num_rows: 9815
     })
     validation_mismatched: Dataset({
         features: ['premise', 'hypothesis', 'label', 'idx'],
         num_rows: 9832
     })
     test_matched: Dataset({
         features: ['premise', 'hypothesis', 'label', 'idx'],
         num_rows: 9796
     })
     test_mismatched: Dataset({
         features: ['premise', 'hypothesis', 'label', 'idx'],
         num_rows: 9847
     })
 }))

In [70]:
huggingface_mnli_dataset['train'][:5]

{'premise': ['Conceptually cream skimming has two basic dimensions - product and geography.',
  'you know during the season and i guess at at your level uh you lose them to the next level if if they decide to recall the the parent team the Braves decide to call to recall a guy from triple A then a double A guy goes up to replace him and a single A guy goes up to replace him',
  'One of our number will carry out your instructions minutely.',
  'How do you know? All this is their information again.',
  "yeah i tell you what though if you go price some of those tennis shoes i can see why now you know they're getting up in the hundred dollar range"],
 'hypothesis': ['Product and geography are what make cream skimming work. ',
  'You lose the things to the following level if the people recall.',
  'A member of my team will execute your orders with immense precision.',
  'This information belongs to them.',
  'The tennis shoes have a range of prices.'],
 'label': [1, 0, 0, 0, 1],
 'idx': [0,

In [73]:
type(huggingface_mnli_dataset['train'][:5]['premise'][0])

str

In [66]:
help(datasets)

Help on package datasets:

NAME
    datasets

DESCRIPTION
    # flake8: noqa
    # coding=utf-8
    # Copyright 2020 The HuggingFace Datasets Authors and the TensorFlow Datasets Authors.
    #
    # Licensed under the Apache License, Version 2.0 (the "License");
    # you may not use this file except in compliance with the License.
    # You may obtain a copy of the License at
    #
    #     http://www.apache.org/licenses/LICENSE-2.0
    #
    # Unless required by applicable law or agreed to in writing, software
    # distributed under the License is distributed on an "AS IS" BASIS,
    # WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
    # See the License for the specific language governing permissions and
    # limitations under the License.

PACKAGE CONTENTS
    arrow_dataset
    arrow_reader
    arrow_writer
    builder
    combine
    commands (package)
    config
    data_files
    dataset_dict
    features (package)
    filesystems (package)
    finger

In [75]:
from transformers.data.processors.utils import DataProcessor, InputExample, InputFeatures

help(DataProcessor)

Help on class DataProcessor in module transformers.data.processors.utils:

class DataProcessor(builtins.object)
 |  Base class for data converters for sequence classification data sets.
 |  
 |  Methods defined here:
 |  
 |  get_dev_examples(self, data_dir)
 |      Gets a collection of :class:`InputExample` for the dev set.
 |  
 |  get_example_from_tensor_dict(self, tensor_dict)
 |      Gets an example from a dict with tensorflow tensors.
 |      
 |      Args:
 |          tensor_dict: Keys and values should match the corresponding Glue
 |              tensorflow_dataset examples.
 |  
 |  get_labels(self)
 |      Gets the list of labels for this data set.
 |  
 |  get_test_examples(self, data_dir)
 |      Gets a collection of :class:`InputExample` for the test set.
 |  
 |  get_train_examples(self, data_dir)
 |      Gets a collection of :class:`InputExample` for the train set.
 |  
 |  tfds_map(self, example)
 |      Some tensorflow_datasets datasets are not formatted the same way t